In [1]:
# 라이브러리 불러오기
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tqdm import tqdm
import time
from label import id2label # Label 정보 매핑용 사전, label.py의 내용을 불러 옴

In [2]:
# NER 모델 및 토크나이저 불러오기
model_name = 'KPF/KPF-BERT-NER'
ner_pipeline = pipeline(
    task='ner',
    model=model_name,
    tokenizer=model_name,
    aggregation_strategy='simple',
    framework='pt'
)

Device set to use mps:0


In [3]:
# 종목명 사전 불러오기
company_df = pd.read_csv('../fastapi/datas/KRX_기업명_20250521.csv')
company_names = set(company_df['종목명'].dropna().unique())

In [4]:
# 뉴스 데이터 불러오기 및 제목 + 본문 결합
news_df = pd.read_csv('../fastapi/datas/mknews_with_id_embedding.csv')
news_df['text_combined'] = (
    news_df['title'].fillna('') + ' ' + news_df['content'].fillna('')
).str.replace(r'\s+', ' ', regex=True).str.strip()

In [5]:
# 텍스트 자르기
def split_text(text, max_length=500):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

In [6]:
# WordPiece 병합 + 라벨 기반 종목 추출
def extract_stock_labels_by_label_id(text):
    chunks = split_text(text, max_length=500)
    matched_names = set()
    
    for chunk in chunks:
        entities = ner_pipeline(chunk)

        # WordPiece 병합 처리
        merged_entities = []
        current_word = ''
        current_score = []
        current_label = ''
        for ent in entities:
            word = ent['word']
            score = ent['score']
            label_id = ent['entity_group']
            label_num = int(label_id.split('_')[1])
            label_name = id2label.get(label_num, "")
            key = label_name[2:] if label_name.startswith(('B-', 'I-')) else label_name
            
            # 경제 조직 라벨로 제한 (OGG_ECONOMY 포함)
            if key.startswith('OGG_ECONOMY'):
                if word.startswith('##'):
                    current_word += word[2:]
                    current_score.append(score)
                else:
                    if current_word:
                        merged_entities.append((current_word, current_label, sum(current_score)/len(current_score)))
                    current_word = word
                    current_score = [score]
                    current_label = label_name
        if current_word:
            merged_entities.append((current_word, current_label, sum(current_score)/len(current_score)))

        for word, label, score in merged_entities:
            if word in company_names:
                matched_names.add(word)

    return list(matched_names)

In [7]:
# 전체 뉴스(9005건)에 적용
results = []
start_time = time.time()

progress = tqdm(enumerate(news_df['text_combined']), total=len(news_df), desc='전체 뉴스 종목명 라벨링 중')

for idx, text in progress:
    try:
        matched = extract_stock_labels_by_label_id(text)
    except Exception as e:
        print(f'오류 (인덱스 {idx}): {e}')
        matched = []
    results.append(matched)

    # 평균 시간 및 남은 시간 계산
    elapsed = time.time() - start_time
    avg_time = elapsed / (idx + 1)
    remaining = avg_time * (len(news_df) - idx - 1)

    # 진행률 바에 부가 정보 추가
    progress.set_postfix({
        '진행': f'{idx+1}/{len(news_df)}',
        '평균': f'{avg_time:.2f}s',
        '남은시간': f'{remaining:.1f}s'
    })

news_df['labels_by_id'] = results

전체 뉴스 종목명 라벨링 중: 100%|██████████| 9005/9005 [16:17<00:00,  9.22it/s, 진행=9005/9005, 평균=0.11s, 남은시간=0.0s]   


In [8]:
# 결과 확인
for i, row in news_df.iterrows():
    print(f'\n=== 뉴스 {i+1} ===')
    print('제목:', row['title'])
    print('출력된 종목:', row['labels_by_id'])


=== 뉴스 1 ===
제목: 배터리도 증시 대피처 아니었다…한달 동안 -23% 빠진 LG엔솔
출력된 종목: ['LG에너지솔루션', 'NH투자증권']

=== 뉴스 2 ===
제목: “돈 못갚는 사람, 돈 못빌리는 사람 는다”…불황주 우울한 급등
출력된 종목: ['미래에셋증권']

=== 뉴스 3 ===
제목: 삼성전자 4분기 실적 ‘한파주의보’…“내년 상반기가 더 문제”
출력된 종목: ['삼성증권', '다올투자증권', 'SK하이닉스', '삼성전자', '유진투자증권']

=== 뉴스 4 ===
제목: 코리아에셋투자증권, 소외 아동 위한 ‘산타 봉사’ 진행
출력된 종목: []

=== 뉴스 5 ===
제목: ‘부동산 스트레스’ 침체 걱정에 뉴욕증시 하락 마감…시장의 눈은 PCE 물가로 [월가월부]
출력된 종목: []

=== 뉴스 6 ===
제목: “새주인을 찾습니다”…케이카, 사흘 연속 상승세
출력된 종목: ['케이카']

=== 뉴스 7 ===
제목: [단독]네이버, 자이언트스텝 지분 블록딜...포쉬마크 인수 대금 마련?
출력된 종목: []

=== 뉴스 8 ===
제목: 씽크풀, ‘2022 벤처창업진흥 유공 포상’ 국무총리 표창 수상
출력된 종목: []

=== 뉴스 9 ===
제목: 한발 물러난 컬리…오아시스, ‘이커머스 1호 상장’ 주인공되나
출력된 종목: ['신세계', '이마트']

=== 뉴스 10 ===
제목: "경기 바닥일때 사자"… 화학·철강株 '활활'
출력된 종목: ['현대제철', 'POSCO홀딩스', '효성티앤씨']

=== 뉴스 11 ===
제목: “강남 용산 빼고 부동산규제 다 푼다는데”…증권가 건설株 ‘갸우뚱’
출력된 종목: ['대우건설', '삼성증권', '현대건설', 'GS건설']

=== 뉴스 12 ===
제목: “땡큐, 폴란드”…해외 수주 기대감에 되살아난 방산주
출력된 종목: ['한국항공우주', '현대로템', '삼성증권', 'SK증권', 'LIG넥스원', '한화에어로스페이스']

=== 뉴스 13 ===
제목: 날개 꺾였던 게